# Final Project CompStats | Timo Haupt | SoSe 2021

---

# Performance of Causal Forests in Identifying Heterogeneous Treatment Effects

---

**Importing Packages:**

In [12]:
suppressMessages(library(grf))
suppressMessages(library(FNN))
suppressMessages(library(MASS))

Warning message:
"package 'grf' was built under R version 3.6.3"

**Importing Functions:**

In [39]:
source("material/auxiliary.R")

## Structure

[**1. Theoretical Discussion**](#theory)
   * [Causal Tree](#tree)
   * [Treatment Effect Estimation](#treatment)
   * [Causal Forest](#forest)
    
[**2. Simulation Study**](#simulation)
   * [Data Generating Process (Treatment Effect Heterogeneity)](#dgp)
   * [Performance Measure](#performance)
   * [Variations](#simstudy)

**3. Application**
   * Data: LaLonde
   * Results Deijiha
   * Results Causal Forest

**4. Conclusion**

[**5. References**](#references)

## 1.) Theoretical Discussion<a id="theory"></a>

This first section introduces the two analyzed estimation methods, Causal Forest and Causal KNN, and describes how treatment estimation is conducted. The main problem in settings dealing with causality and inference is the missing counterfactual. Particularly in estimating treatment effects it's only possible to observe an individual with treatment, $Y_i^{(1)}$, or without treatment, $Y_i^{(0)}$, but never both. Therefore, we are in the Potential Outcome Framework where we "posit the existence of potential outcomes $Y_i^{(1)}$ and $Y_i^{(0)}$ corresponding respectively to
the response the $i$-th subject would have experienced with and without the treatment" (Athey/Wager, 2015).

Both introduced methods, Causal Forest and Causal KNN, try to find these potential outcomes by some sort of matching algorithm in order to estimate the treatment effect 
$$\tau(x) = \mathbb{E}\left[Y_i^{(1)} - Y_i^{(0)} \mid X_i = x\right] $$

One essential assumption we need to impose in order to theoretically justify both methods is unconfoundedness, i.e. "treatment assignment $W_i$ is independent of the potential outcomes for $Y_i$ conditional on $X_i$" (Athey & Wager 2015).

$$\left \{Y_i^{(1)}, Y_i^{(0)} \right \} \perp\!\!\!\perp W_i \mid X_i $$

The idea is that, if unconfoundedness holds, "we can treat nearby observations in x-space as having come from a
randomized experiment; thus, nearest-neighbor matching and other local methods will in general be consistent for $\tau(x)$" (Athey & Wager 2015).

### Causal Tree<a id="tree"></a>

* Causal Trees are very similiar to Random Trees. 

* Difference: rather than predicting outcome $Y$, they predict treatment effects

* "splitting rule that maximizes i2Rk ^ (Xi)2, which maximizes the variance of the predicted treatment effects ^ (Xi) across the observations in the two new leaves. This splitting rule is feasible given the data and mimics the approach that would be used if the treatment effect was observed." (Hitsch Misra)

* "the algorithm seeks to maximize the treatment effect heterogeneity across partitions at every tree-splitting step" (Farbmacher)



### Treatment Effect Estimation<a id="treatment"></a>


Let's take some $x \in R_k$ where $R_k$ is some leaf of the Causal Forest and define $N_k(w)$ as the number of observations in the leaf $R_k$ with treatment status $w \in \{0,1\}$. The Conditional Average Treatment Effect is then estimated "based on the mean difference between the outcome levels of the treated and untreated units" which are in the same leaf $R_k$ by 

$$ \hat{\tau}(x) = \hat{\tau}_{R_k} = \frac{1}{N_k(1)} \sum_{i \in R_k(1)} Y_i-\frac{1}{N_k(0)} \sum_{i \in R_k(0)} Y_i  $$
So, the Causal Forest does some sort of matching by recursively partitioning the predictor space and then matching treated and untreated observations that have fallen in the same leaf (see e.g. Hitsch & Misra 2018 and Athey & Wager 2015).

* Basically, we have for each individual $i$ with characteristics $x$ some specific treatment effect. 


### Causal Forest<a id="forest"></a>

* "Finally, given a procedure for generating a single causal tree, a causal forest generates an ensemble of B such trees, each of which outputs an estimate $\hat{\tau}_b(x)$. The forest then aggregates their predictions by averaging them: $\hat{\tau}(x) = \frac{1}{B} \sum_{b=1}^B \hat{\tau}_b(x)$" (Athey Wager 2015)

* I will go with the default of $B=2000$ causal trees per causal forest.

* Advantage of Causal Forest: "In practice, this aggregation scheme helps reduce variance and smooths sharp decision boundaries" (Athey Wager 2015)

**Honest Splitting Rule**

* "Double-sample trees split the available training data into two parts: one half for estimating the desired response inside each leaf, and another half for placing splits."

* "Double-sample causal trees are defined similarly, except that for prediction we estimate ^ (x) using (5) on the I sample. Following Athey and Imbens [2015], the splits of the tree are chosen by maximizing the variance of $\tau(Xi)$"

* "Then, it uses the J -sample to place the splits, while holding out the I-sample to do within-leaf estimation"

Athey & Wager (2015) Chapter 2.4

**Advantage of CF**

"At a high level, trees and forests can be thought of as nearest neighbor methods with an
adaptive neighborhood metric. Given a test point x, classical methods such as k-nearest
neighbors seek the k closest points to x according to some pre-specied distance measure,
e.g., Euclidean distance. In contrast, tree-based methods also seek to nd training examples
that are close to x, but now closeness is dened with respect to a decision tree, and the
closest points to x are those that fall in the same leaf as it. The advantage of trees is that
their leaves can be narrower along the directions where the signal is changing fast and wider
along the other directions, potentially leading a to a substantial increase in power when the
dimension of the feature space is even moderately large." 

"Our causal forest estimator can be thought of as an adaptive nearest
neighbor method, where the data determines which dimensions are most important to consider
in selecting nearest neighbors. Such adaptivity seems essential for modern large-scale
applications with many features."(Athey & Wager 2015)

"When estimating a random forest, we provide the algorithm with the set of features and the
treatment indicator, Wi. Specifying the interactions between Xi and Wi is not be necessary
because random forests, a non-parametric estimation method, in principle should be able to
automatically detect any relevant interactions." (Farbmacher et al.)

#### GRF package

**Plot example Causal Tree**

# 2.) Simulation Study<a id="simulation"></a>

## Data Generating Process<a id="dgp"></a>

Note that the codes for the Data Generating Process and the Monte Carlo Simulation Study can be found in the auxiliary-file. The variations in the treatment effect function $\tau(x)$ and the DGP function are inspired by Athey & Wager (2015) and Powers et al. (2017). 


$$X_{i,1}, ..., X_{i,10} \sim \mathcal{N}(0,1)$$

$$\beta \in [0.5,1]^{10}$$

$$Prob(W_i=1) =0.5$$

$$Y_i = X \beta + \tau(x_i)* W_i + \epsilon_i $$

$$\epsilon_i \sim \mathcal{N}(0,\,0.1)$$

**Description**

The specification above displays the basic setup of the Data Generating Process. 
10 regressors are drawn from a Standard Normal Distrubtion and all $\beta$ are evenly divided between 0.5 and 1. 
The probability of being assigned to treatment is fixed at 50% for all observations. The individual error term is also drawn from a Normal Distribution with mean zero and standard deviation of 0.1.
All changes and additions that have been made in the Simulation Study, will be explained in greater detail as a description before each scenario.

## Performance Measure<a id="performance"></a>

Eventhough the counterfactuals are never observed in real life data, we can take advantage of the Data Generating Process in Simulation Studies where we explicitely calculate the true treatment effect for each observation. Therefore, as Athey & Wager (2015) proposed we can use the $MSE_{\tau}$ as a measure in order to compare the performance of the Causal Forest in different settings. I calculated the Mean Squared Error of the treatment effect as follows
$$MSE_{\tau} = \frac{1}{N} \sum_{i=1}^N{\left( \tau_{i}-\hat{\tau}_{R_k}(X_i) \right)^{2}} $$

, where $\hat{\tau}_{R_k}(X_i)$ are the estimated treatment effects for individuals in leaf $R_k$.
Note that the $MSE_{\tau}$ used in this Simulation Study differs slightly from the one proposed in the Athey and Imbens (2015) paper. They subtract $\mathbb{E}[\tau_i^2]$, but as they noted "it does not affect how the criterion ranks estimators", since $\tau_i$ is not a Random Variable. 

## Monte Carlo Simulation Study - Variations<a id="simstudy"></a>

### Case 1(new) - Introduction of Treatment Effect Functions

$(1)\quad \tau(x) = 0.2 $

$(2)\quad \tau(x) = 0.1 + 0.1*\mathbb{1}(x_1 > 0) + 0.1*\mathbb{1}(x_2 > 0)$

$(3)\quad \tau(x) = 0.1 + x_1*\mathbb{1}(x_1 > 0) + x_2*\mathbb{1}(x_2 > 0)$  

$(4)\quad \tau(x) = 0.1 + \sum_{i=1}^{4} \mathbb{1}(x_i > 0)*x_i $

$(5)\quad \tau(x) = 0.2 + x_1*x_2 $


**Description & Result**

Case 1 will be used as a benchmark where the basic setup is considered. The different treatment scenarios are depicted above. 
In the first scenario $(1)$ there is only a single treatment effect independent of individuals' characteristics. Treatment scenario $(2)$ introduces some simple treatment effect heterogeneity, that is treatment effect increases if an individual has positive values in its' $X_1$ and $X_2$ characteristics respectively. Each positive value increases the magnitude by the mean treatment effect of 0.1. 
In treatment scenario $(3)$ complexity of heterogeneity is increased, such that the magnitude of treatment depends on the values of $X_1$ and $X_2$. So, treatment effect is even heterogeneous within $X_1$ and $X_2$. Higher positive values of these variables imply a higher treatment effect. 
Scenario $(4)$ adds two more regressors in the treatment effect function. 
Scenario $(5)$ introduces an interaction term in $\tau(x)$. Since $X$ are standard normally distributed, around half of the draws from $X_1$ and $X_2$ are negative, implying an even higher degree of treatment variation.

It can be observed that the higher the number of observations, the smaller the Mean Squared Error. This holds true for all successive applications. This is theoretically justified, since with a higher number of observations the trees can perform more splits per tree without reaching the boundary of minimal observations per leaf. 
Interesting is that the $MSE_{\tau}$ almost doesn't differ between homogeneous and a very simple treatment effect, $(1)$ and $(2)$. This result is quite remarkable and shows that Causal Forests seem to be quite good at matching observations with a simple heterogeneity structure. It should be noted that Powers et al. (2017) find some striking differences between scenarios 1 and 2 in their paper which come closest to my Treatment Scenarios $(1)$ and $(2)$. One reason might be that Powers et al. (2017) draw their regressors mixed from Standard Normal and Bernoulli distributions.
Since Scenarios $(3)$ to $(4)$ imply a higher degree of variation in individual treatment effects, it is obvious that the $MSE_{\tau}$ are much higher compared to the previous scenarios. Even for a large sample the estimated $MSE_{\tau}$ can't keep up. These results are consistent though with the ones from Table S1 (Appendix) in Athey and Imbens (2015).

In [40]:
set.seed(123)

In [41]:
simulation(p=10,numsim=100,regressors="nocorr")

Application,N = 100,N = 150,N = 250,N = 500
Treatment 1,0.270,0.140,0.098,0.059
Treatment 2,0.203,0.152,0.109,0.081
Treatment 3,0.833,0.698,0.565,0.382
Treatment 4,1.423,1.271,1.061,0.784
Treatment 5,1.188,1.167,1.061,0.996


### Case 2(new) - Polynomials and Interactions added to DGP

$(1) \quad X_{i,9} = X_{i,1}^2 \quad $ and $\quad X_{i,10} = X_{i,2}^2$

$(2) \quad X_{i,2} = X_{i,1}^2$

$(3) \quad X_{i,10} = X_{i,1}*X_{i,2}$

**Description & Result**

In Case 2 different changes in the Data Generating Process have been made. In scenario $(1)$ quadratic transformations of $X_1$ and $X_2$ have been added in the DGP function. Scenario $(2)$ differs in that the quadratic transformation of $X_1$ displaces the $X_2$ variable, implying that only $X_1$ and it's second degree polynomial determine treatment heterogeneity. $(3)$ depicts the scenario with an additional interaction term between $X_1$ and $X_2$ in the DGP.



Causal Forests seem to make pretty good treatment predictions compared to Case 1 without any additional interactions or polynomials. Since it's a non-parametric method and therefore don't make explicit assumptions about the underlying functional form, this result seems to be consistent with the theoretical properties.
Adding the interaction term between $X_1$ and $X_2$ into the DGP makes the Causal Forest perform the best within all scenarios where polynomials or interactions have been considered. 
As Hitsch and Misra (2018) state it, this makes sense, since Causal Forest is "a non-parametric estimation method, [and] in principle should be able to automatically detect any relevant interactions".
The good performance compared to Case 1 could also be explained by the fewer number of total regressors, since either one or two regressors basically include the same information as $X_1$ and/or $X_2$. The performance with fewer regressors will also be discussed in Case 5.

In [42]:
simulation(p=10,numsim=100,regressors="poly1")

Application,N = 100,N = 150,N = 250,N = 500
Treatment 1,0.237,0.187,0.117,0.072
Treatment 2,0.194,0.178,0.103,0.072
Treatment 3,0.919,0.742,0.539,0.360
Treatment 4,1.517,1.360,1.087,0.783
Treatment 5,1.307,1.208,1.104,0.988


In [43]:
simulation(p=10,numsim=100,regressors="poly2")

Application,N = 100,N = 150,N = 250,N = 500
Treatment 1,0.233,0.152,0.111,0.073
Treatment 2,0.227,0.150,0.120,0.068
Treatment 3,3.037,2.454,1.534,0.850
Treatment 4,3.614,3.075,2.305,1.343
Treatment 5,13.124,12.619,10.595,7.334


In [44]:
simulation(p=10,numsim=100,regressors="interaction")

Application,N = 100,N = 150,N = 250,N = 500
Treatment 1,0.206,0.157,0.099,0.076
Treatment 2,0.294,0.194,0.118,0.073
Treatment 3,0.835,0.683,0.512,0.348
Treatment 4,1.517,1.321,1.025,0.750
Treatment 5,1.031,0.926,0.715,0.481


### Case 3 - Correlation between Regressors

$(1) \quad \rho_{j,k} = 0.5 \qquad \forall j,k = \{1,...,10\} \quad s.t. \quad j \ne k  $

$(2) \quad \rho_{1,2} = 0.9$

**Description & Result**

Case 3 deals with correlation between regressors. In scenario $(1)$ all regressors are moderately correlated with each other by $\rho = 0.5$ and in $(2)$ only the two regressors that influence treatment heterogeneity are highly correlated by $\rho = 0.9$. This exploration might be interesting for economic settings with some sorts of trends, where variables have some influence of each other.

Except for the first treatment setting, in both correlation scenarios Causal Forest performs a little bit wors for small samples, but improves significantly with large samples. These findings are interesting in that Matching Estimators, Causal Forest in particular, seem to handle the problem of correlation between regressors very well, especially in large samples. **Theoretical Explanation?**

In [48]:
simulation(p=10,numsim=100,regressors="corr1")

Application,N = 100,N = 150,N = 250,N = 500
Treatment 1,0.315,0.201,0.103,0.060
Treatment 2,0.287,0.177,0.109,0.065
Treatment 3,1.096,0.827,0.579,0.365
Treatment 4,2.538,1.850,1.334,0.775
Treatment 5,1.523,1.323,1.145,0.913


In [ ]:
simulation(p=10,numsim=100,regressors="corr2")

### Case 4 - Treatment variables don't influence Y

$(1) \quad \beta_1, \beta_2 = 0$

**Description & Result**

In Case 2 both $X_1$ and $X_2$ influence treatment effect heterogeneity, but don't have any influence on the output variable $Y$. Since the number of regressors has an impact of how well the estimation method performs, I conduct this analysis with 12 regressors (minus the two regressors with $\beta = 0$) in order to assure comparability to Case 1 with 10 regressors in total. 

Causal Forests perform very similiar to the scenario where $\beta_1, \beta_2 \neq 0$, indicating that they are able to detect heterogeneity based on variables that don't influence $Y$. This makes sense from a theoretical point of view, since the splits of the tree are being made as to generate some high level of treatment variation between the split directions. Therefore, detection of treatment heterogeneity works as long as these variables are included in the dataset, eventhough they don't influence $Y$ directly. 

In [ ]:
simulation(p=10,numsim=100,regressors="beta12")

### Case 5 - Treatment Probability depends on $X$

$(1) \quad Prob(W_i = 1 \mid x_1>0) = 0.75 \quad $ and $\quad Prob(W_i = 1 \mid x_1<0) = 0.25 $

**Description & Result**

Before Marginal Treatment Probability has been fixed at 0.5 for all individuals. In this scenario Selection Bias will be introduced. Those individuals profiting more from Treatment, i.e. individuals with positive values in the $X_1$ characteristic, are more likely to be assigned to Treatment. Treatment Probability for $X_1 > 0$ is 0.75, whereas Treatment Probability for $X_1 < 0$ is only 0.25. The analysis of this case has importance in observational studies where randomization in treatment assignment can often not be assured.  

We see a slight increase in $MSE_{\tau}$ compared to Case 1 for 100 and 150 observations, but Causal Forests perform relatively worse for 500 observations, indicating that the performance with selection bias cannot quite keep up in large scale samples, where both methods have usually their largest improvements, as seen in previous cases.

In [ ]:
simulation(p=10,numsim=100,regressors="selectionbias")

### Case 6 - Number of Regressors

$(1)-(4) \quad p \in \{4,20,50,100\}$

**Description & Result**

In this final scenario performance with a different number of regressors is compared. Number of regressors is varied between 4 and 100. This comparison is interesting in applications where the researcher might have a dataset containing a lot of different characteristics. 

I find that the overall performance decreases if more regressors are added. An interesting finding is that the KNN methods can't really improve their performance over the number of observations, whereas Causal Forest signficantly decreases $MSE_{\tau}$ going from 100 to 500 observations. So, even if one has a high dimensional data set, this simulation study shows that if the number of observations is quite large the Causal Forest will perform quite well. THe relatively well performance of the Causal Forest in high dimensional space can be explained theoretically by it's variable selection feature. By training the trees with only the most "important" variables, only a couple variables will be selected by the Causal Tree overall. On the other hand, KNN tries to find the closest neighbors in the Euclidean Space and is therefore not reducing dimensionality. 

* Economic Setting: Higher Dimensional Datasets, Purpose?

* **Warum wird die Treatment Vorhersage schlechter je mehr Regressoren verwendet werden???**

* **Intuition**

* CF auch im high dimensional space better, since it performs a kind of variable selection, by training the trees with only the most important variables!

* no big increase in MSE going from 50 to 100 regressors. Good sign for Application purposes!

In [ ]:
simulation(p=4,numsim=100,regressors="nocorr")

In [36]:
#### NEW

simulation(p=20,numsim=10,regressors="nocorr")

Application,N = 100,N = 150,N = 250,N = 500
Treatment 1,0.541,0.425,0.204,0.122
Treatment 2,0.479,0.372,0.221,0.100
Treatment 3,0.881,0.865,0.780,0.611
Treatment 4,2.343,1.550,1.356,1.217
Treatment 5,1.362,1.147,1.130,1.123


In [37]:
#### NEW

simulation(p=50,numsim=10,regressors="nocorr")

Application,N = 100,N = 150,N = 250,N = 500
Treatment 1,0.665,0.859,0.625,0.297
Treatment 2,0.550,0.707,0.663,0.367
Treatment 3,1.808,1.498,1.087,0.807
Treatment 4,3.714,2.517,1.898,1.538
Treatment 5,1.913,1.932,1.851,1.318


In [38]:
#### NEW

simulation(p=100,numsim=10,regressors="nocorr")

Application,N = 100,N = 150,N = 250,N = 500
Treatment 1,1.209,2.166,1.225,0.473
Treatment 2,1.431,2.530,1.530,0.616
Treatment 3,1.917,2.186,2.335,1.041
Treatment 4,4.147,3.515,1.880,1.513
Treatment 5,4.130,2.020,1.972,1.445


In [47]:
#CF_simstudy(p=10,N=500,numsim=50,dgp2)

In [46]:
#CF_simstudy(p=10,N=500,numsim=10,dgp3)

In [45]:
#CF_simstudy(p=10,N=500,numsim=10,dgp4)

## References <a id="references"></a>

* **Athey & Imbens (2015).** [Recursive partitioning for heterogeneous causal effects](https://www.pnas.org/content/113/27/7353). Colloquium Paper.


* **Athey & Wager (2015).** [Estimation and Inference of Heterogeneous Treatment Effects using Random Forests](https://arxiv.org/abs/1510.04342). 


* **Athey & Wager (2019a).** [Estimating Treatment Effects with Causal Forests: An Application](https://www.gsb.stanford.edu/faculty-research/working-papers/estimating-treatment-effects-causal-forests-application). Working Paper No. 3786. 


* **Athey, Wager, Hadad, Klosin, Muhelbach, Nie & Schaelling (2020).** [Tutorial: Estimation of Heterogeneous Treatment Effects](https://gsbdbi.github.io/ml_tutorial/hte_tutorial/hte_tutorial.html). Tutorial for “Machine Learning and Causal Inference” class.


* **Farbmacher, Kögel & Spindler (2019).** [Heterogeneous Effects of Poverty on Cognition](https://www.mpisoc.mpg.de/en/social-policy-mea/publications/detail/publication/heterogeneous-effects-of-poverty-on-cognition/). MEA Discussion Paper (06-2019).


* **Hitsch & Misra (2018).** [Heterogeneous Treatment Effects and Optimal Targeting Policy Evaluation](https://papers.ssrn.com/sol3/papers.cfm?abstract_id=3111957). Randomized Social Experiments eJournal (2018).


* **Powers, Qian, Jung, Schuler, Shah, Hastie & Tibshirani (2017).** [Some methods for heterogeneous treatment effect estimation in high-dimensions](https://web.stanford.edu/~hastie/Papers/PM_Powers_SIM.pdf).Stat Med. 2018 May 20;37(11):1767-1787. 